In [ ]:
#Instalación de la librería gseapy
pip install gseapy

In [ ]:
#Instalación de la librería matplotlib
pip install matplotlib

In [ ]:
#Instalación de wget
pip install wget

In [ ]:
#Importo las librerías a usar
import pandas as pd
import wget
import matplotlib.pyplot as plt
import numpy as np
import gseapy as gp




In [ ]:
#Descargo el set de datos
url='https://raw.githubusercontent.com/jfmaggio/tp_final_programacion/main/GSE113469.top.table.csv'

wget.download(url)

In [ ]:
#Los datos descargados los cargo en un dataframe


genes_principal = pd.read_csv('GSE113469.top.table.csv', on_bad_lines='skip')
print(genes_principal.head(n=25)) #Imprimo las primeras 25 lineas del dataframe


In [ ]:
#Averigo el rango de valores de los datos
print('El mínimo de valores de la columna adj.P.Value es:', genes_principal['adj.P.Val'].min(), 'y el máximo es: ', genes_principal['adj.P.Val'].max())
print('El mínimo de valores de la columna logFC es:', genes_principal['logFC'].min(), 'y el máximo es: ', genes_principal['logFC'].max())
print('El mínimo de valores de la columna GI es:', genes_principal['GI'].min(), 'y el máximo es: ', genes_principal['GI'].max())


In [ ]:
#Para ver que tipo de datos tiene el dataframe
genes_principal.map(type).apply(set)

In [ ]:
genes_principal['ID'].count() 

In [ ]:
genes_principal['Gene.symbol'].isnull().sum() #Chequeo cuantos nombres de genes faltan

In [ ]:
genes_principal['ID'].count()#Chequeo cuantos nombres de genes hay

In [ ]:
#Saco las filas con NaN
genes_principal = genes_principal.dropna(subset=['Gene.symbol'])

In [ ]:
genes_principal['ID'].count()#Chequeo cuantos nombres de genes hay

In [ ]:
print(genes_principal.head(n=25)) #Imprimo las primeras 25 lineas del dataframe

In [ ]:
#Creo un histograma con las frecuencias de los valores p ajustados
plt.hist(genes_principal['adj.P.Val'], bins=100, edgecolor='black')


plt.xlabel('Valor p Ajustado')
plt.ylabel('Frecuencia')
plt.title('Histograma de Valor p Ajustado')
plt.savefig( 'Histograma.png', bbox_inches='tight')

In [ ]:
#Se crea un grafico de volcan para visualizar los genes diferencialmente expresados, y se colocan ineas delimiando los que seran filtrados
# Separo los genes sobreexpresados y subexpresados respecto de los controles
sob = genes_principal.loc[genes_principal['logFC'] > 0]
sub = genes_principal.loc[genes_principal['logFC'] < 0]

#Calculo el -log(p austado)
neg_logpvalue_sob = -np.log10(sob['adj.P.Val'])
neg_logpvalue_sub = -np.log10(sub['adj.P.Val'])


# Dibujar los genes sobreexpresados y subexpresados con colores diferentes
plt.scatter(sob['logFC'], neg_logpvalue_sob, color='green', label='Sobreexpresados', alpha=1, s=5)
plt.scatter(sub['logFC'], neg_logpvalue_sub, color='blue', label='Subexpresados', alpha=1, s=5)

# Establecer los límites y etiquetas de los ejes
plt.xlim(-5, 5)

plt.xlabel('Log Fold Change')
plt.ylabel('-log10(p-value)')
plt.title('Gráfico de Volcán')

# Añado una línea para resaltar los genes significativamente diferencialmente expresados
threshold = -np.log10(0.05)
plt.axhline(y=threshold, color='red', linestyle='-', linewidth=0.5)

# Añado líneas verticales para resaltar los genes diferencialmente expresados con |logFC| > 1
plt.axvline(x=1, color='red', linestyle='--', linewidth=1)  # Ejemplo de línea vertical en x=1
plt.axvline(x=-1, color='red', linestyle='--', linewidth=1)  # Ejemplo de línea vertical en x=-1


# Añadir leyenda
plt.legend()

plt.savefig( 'graf_volcan.png', bbox_inches='tight')

In [ ]:
%load_ext autoreload
%autoreload 2
#Importo las librerías
import pandas as pd
import gseapy as gp


In [ ]:
#Filtro por adj.P.Val, los menores a 0.05
genes_principal = genes_principal[genes_principal['adj.P.Val'] < 0.05]
#Cuento cuantos genes quedan
print("Después del filtrado quedan", genes_principal['Gene.symbol'].count(), "genes")

In [ ]:
#Ahora armo dos dataframes, uno con los genes sobreexpresados con expresión ayor a 1 (logFC > 1) y el otro con los subexpresados con expresión menor a -1 (logFC < (-1)).
genes_sob_mayoresa1 = genes_principal[genes_principal['logFC'] > 1]

genes_sub_menoresa1 = genes_principal[genes_principal['logFC'] < (-1)]

#Cuento cuantos genes quedan
print("Después del filtrado por LogFC quedan", genes_sob_mayoresa1['Gene.symbol'].count(), "genes sobreexpresados y ", genes_sub_menoresa1['Gene.symbol'].count(), "genes subexpresados respecto de los controles.")


In [ ]:
#Checkeo los dataframes creados
genes_sob_mayoresa1.head()

In [ ]:
#Checkeo los dataframes creados
genes_sub_menoresa1.head()

In [ ]:
#Ya habiendo hecho los filtrados correspondientes, hago el análisis de enriquecmiento de los genes sobreexpresados respecto de los controles.
enr_sob = gp.enrichr(gene_list=genes_sob_mayoresa1['Gene.symbol'], 
                 gene_sets=['GO_Biological_Process_2023','GO_Cellular_Component_2023', 'GO_Molecular_Function_2023', 'KEGG_2021_Human'],
                 organism='human', 
                 outdir=None,
                )

In [ ]:
#Importo la funciones de gsapy para graficar
from gseapy import barplot, dotplot

In [ ]:
#Se grafican los resultados obtenidos para los genes sobreexpresados, usando un gráfico de puntos
ax_sob = dotplot(enr_sob.results,
              column="Adjusted P-value",
              x='Gene_set', 
              size=5,
              top_term=3,
              figsize=(3,5),
              title = "Analisis de Enriquecimiento de Genes Sobreexpresados",
              xticklabels_rot=45,
              show_ring=True, 
              marker='o',
             )
plt.savefig( 'puntos_sob.png', bbox_inches='tight')

In [ ]:
#Se grafican los resultados obtenidos para los genes sobreexpresados, usando un gráfico de barras
bar_sob = barplot(enr_sob.results,
              column="Adjusted P-value",
              group='Gene_set',
              size=5,
              top_term=3,
              figsize=(3,5),
              title = "Analisis de Enriquecimiento de Genes Sobreexpresados",
              color = {'GO_Biological_Process_2023': 'cornflowerblue', 'GO_Cellular_Component_2023':'darkblue', 'GO_Molecular_Function_2023':'lightskyblue', 'KEGG_2021_Human':'green'}
             )
plt.savefig( 'barras_sob.png', bbox_inches='tight')

In [ ]:
enr_sob.results

In [ ]:
enr_sob.results=enr_sob.results.sort_values(by = "Adjusted P-value")

enr_sob.results[['Gene_set', 'Term', 'Overlap', 'Adjusted P-value']].head(n=25)


In [ ]:
enr_sob.results[enr_sob.results['Gene_set'] == "GO_Biological_Process_2023"].head(n=3)

In [ ]:
enr_sob.results[enr_sob.results['Gene_set'] == "GO_Cellular_Component_2023"].head(n=3)

In [ ]:
enr_sob.results[enr_sob.results['Gene_set'] == "GO_Molecular_Function_2023"].head(n=3)

In [ ]:
enr_sob.results[enr_sob.results['Gene_set'] == "KEGG_2021_Human"].head(n=3)

In [ ]:
#Ya habiendo hecho los filtrados correspondientes, hago el análisis de enriquecmiento de los genes subexpresados respecto de los controles.
enr_sub = gp.enrichr(gene_list=genes_sub_menoresa1['Gene.symbol'], 
                 gene_sets=['GO_Biological_Process_2023','GO_Cellular_Component_2023', 'GO_Molecular_Function_2023', 'KEGG_2021_Human'],
                 organism='human', 
                 outdir=None,
                )

In [ ]:
#Se grafican los resultados obtenidos para los genes subexpresados, usando un gráfico de puntos
ax_sub = dotplot(enr_sub.results,
              column="Adjusted P-value",
              x='Gene_set', 
              size=20,
              top_term=3,
              figsize=(3,5),
              title = "Analisis de Enriquecimiento de Genes Subexpresados",
              xticklabels_rot=45,
              show_ring=True, 
              marker='o',
             )
plt.savefig( 'puntos_sub.png', bbox_inches='tight')

In [ ]:
#Se grafican los resultados obtenidos para los genes subexpresados, usando un gráfico de barras
bar_sub = barplot(enr_sub.results,
              column="Adjusted P-value",
              group='Gene_set',
              size=5,
              top_term=3,
              figsize=(5,5),
              title = "Analisis de Enriquecimiento de Genes Subexpresados",
              color = {'GO_Biological_Process_2023': 'cornflowerblue', 'GO_Cellular_Component_2023':'darkblue', 'GO_Molecular_Function_2023':'lightskyblue', 'KEGG_2021_Human':'green'}
             )
plt.savefig( 'barras_sub.png', bbox_inches='tight')

In [ ]:
enr_sub.results.head()

In [ ]:
enr_sub.results=enr_sub.results.sort_values(by = "Adjusted P-value")

enr_sub.results[enr_sub.results['Gene_set'] == "GO_Cellular_Component_2023"].head(n=3)

In [ ]:
enr_sub.results[enr_sub.results['Gene_set'] == "GO_Biological_Process_2023"].head(n=3)

In [ ]:
enr_sub.results[enr_sub.results['Gene_set'] == "GO_Molecular_Function_2023"].head(n=3)

In [ ]:
enr_sub.results[enr_sub.results['Gene_set'] == "KEGG_2021_Human"].head(n=3)